# English Gender Bias Examples

The following example demonstrates how to use any sample gender query on any word embedding model considering a fairness metric. The workflow may be broken 
down into three steps which are:

• Download and install the word embedding model in any desired language.
• Structure the query based on the target set and the attribute set for that particular language using google translator.
• Execute the queries utilizing the fairness metric through the Word Embedding Model.

### How to interpret the results

Any score in WEAT, WEAT-EZ and RNSB metrics greater than 0 suggests that there is indeed a bias for the query in consideration for the particular language. On the other hand, when using ECT metric values closer to 1 are better as they represent less bias.

### Notes about the models and `flair`

In majority of the cases, a score will be achieved. Nevertheless, in a few occurrences, certain queries will contain about 20% missing words and the results will 
reflect that and the query declares itself invalid and returns NaN. In such cases, a “lost vocabulary threshold” parameter is introduced which can limit the number
of words lost or missing. A 30% loss is permitted in the following scenario:


These examples are based on embeddings trained by the authors of the flair library.
To learn more about the library, visit their official website: https://github.com/flairNLP/flair

Also, visit https://github.com/flairNLP/flair/blob/master/resources/docs/embeddings/CLASSIC_WORD_EMBEDDINGS.md
to check the list of embeddings in other languages.

## Preparing the Model and the Queries 

In [ ]:
# use the following code to install/update flair.
!pip install -U flair

In [ ]:
%cd ../..

In [ ]:
from flair.embeddings import WordEmbeddings

from wefe.word_embedding_model import WordEmbeddingModel
from wefe.query import Query
from wefe.datasets import load_weat
from wefe.metrics import RNSB, WEAT, ECT
from wefe.utils import run_queries, plot_queries_results

### Load the model

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

def load_flair_and_create_keyed_vector(flair_model_name):
    # load model from flair
    glove_embedding = WordEmbeddings(flair_model_name)

    # hack to transform pytorch embedding to gensim's KeyedVectors
    keyed_vectors = KeyedVectors(vector_size=glove_embedding.embedding_length)
    keyed_vectors.add_vectors(
        keys=list(glove_embedding.vocab.keys()),
        weights=glove_embedding.embedding.weight.numpy()[:-1, :],
    )

    return keyed_vectors

In [ ]:
flair_model_name = "es"
model = load_flair_and_create_keyed_vector(flair_model_name)
wefe_model = WordEmbeddingModel(model, flair_model_name)

# we will test using only one model. You could add more later...
models = [wefe_model]

### Load word sets

In [ ]:
# load the WEAT word sets
word_sets = load_weat()


# Create gender queries
gender_query_1 = Query(
    [word_sets["male_terms"], word_sets["female_terms"]],
    [word_sets["career"], word_sets["family"]],
    ["Male terms", "Female terms"],
    ["Career", "Family"],
)

gender_query_2 = Query(
    [word_sets["male_terms"], word_sets["female_terms"]],
    [word_sets["science"], word_sets["arts"]],
    ["Male terms", "Female terms"],
    ["Science", "Arts"],
)

gender_query_3 = Query(
    [word_sets["male_terms"], word_sets["female_terms"]],
    [word_sets["math"], word_sets["arts_2"]],
    ["Male terms", "Female terms"],
    ["Math", "Arts2"],
)


gender_queries = [gender_query_1, gender_query_2, gender_query_3]

## Run the Queries

### Run the queries using WEAT


In [ ]:
weat = WEAT()

WEAT_gender_results = run_queries(
    WEAT,
    gender_queries,
    models,
    lost_vocabulary_threshold=0.3,
    metric_params={"preprocessors": [{"lowercase": True}]},
    aggregate_results=True,
    queries_set_name="Gender Queries",
)

display(WEAT_gender_results)
plot_queries_results(WEAT_gender_results).show()

### Run the queries using WEAT effect size


In [ ]:
WEAT_EZ_gender_results = run_queries(
    WEAT,
    gender_queries,
    models,
    lost_vocabulary_threshold=0.3,
    metric_params={"preprocessors": [{"lowercase": True}], "return_effect_size": True,},
    aggregate_results=True,
    queries_set_name="Gender Queries",
)

display(WEAT_EZ_gender_results)
plot_queries_results(WEAT_EZ_gender_results).show()

### Run the queries using RNSB


In [ ]:
RNSB_gender_results = run_queries(
    RNSB,
    gender_queries,
    models,
    lost_vocabulary_threshold=0.3,
    metric_params={"preprocessors": [{"lowercase": True}]},
    aggregate_results=True,
    queries_set_name="Gender Queries",
)
display(RNSB_gender_results)
plot_queries_results(RNSB_gender_results).show()


### Run the queries using ECT


In [ ]:
ECT_gender_results = run_queries(
    ECT,
    gender_queries,
    models,
    lost_vocabulary_threshold=0.3,
    metric_params={"preprocessors": [{"lowercase": True}]},
    aggregate_results=True,
    queries_set_name="Gender Queries",
    generate_subqueries=True
)

display(ECT_gender_results)
plot_queries_results(ECT_gender_results).show()
